In [1]:
from pymor.basic import *
from pymor.core.logger import set_log_levels
set_log_levels({'pymor.algorithms.greedy': 'ERROR', 'pymor.algorithms.gram_schmidt.gram_schmidt': 'ERROR', 'pymor.algorithms.image.estimate_image_hierarchical': 'ERROR'})

In [2]:
p = thermal_block_problem(num_blocks=(3, 2))

In [3]:
fom, fom_data = discretize_stationary_cg(p, diameter=1./50.)

Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

In [4]:
print(fom_data['grid'])

Tria-Grid on domain [0,1] x [0,1]
x0-intervals: 50, x1-intervals: 50
elements: 10000, edges: 15100, vertices: 5101


In [5]:
U = fom.solve([1.0, 0.1, 0.3, 0.1, 0.2, 1.0])
fom.visualize(U, title='Solution')

Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

ThreeJSPlot(children=(HBox(children=(Renderer(children=(Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, …

In [6]:
print(fom.parameters)

{diffusion: 6}


In [7]:
reductor = CoerciveRBReductor(
    fom,
    product=fom.h1_0_semi_product,
    coercivity_estimator=ExpressionParameterFunctional('min(diffusion)', fom.parameters)
)

In [8]:
training_set = p.parameter_space.sample_uniformly(4)
print(training_set[0])

{diffusion: [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]}


In [9]:
greedy_data = rb_greedy(fom, reductor, training_set, max_extensions=32)

Accordion(children=(HTML(value='', layout=Layout(height='16em', overflow_y='auto', width='100%')),), selected_…

In [10]:
print(greedy_data.keys())

dict_keys(['max_errs', 'max_err_mus', 'extensions', 'time', 'rom'])


In [11]:
rom = greedy_data['rom']

In [12]:
RB = reductor.bases['RB']
print(type(RB))
print(len(RB))
print(RB.dim)

<class 'pymor.vectorarrays.numpy.NumpyVectorArray'>
32
5101


In [13]:
import numpy as np
gram_matrix = RB.gramian(fom.h1_0_semi_product)
print(np.max(np.abs(gram_matrix - np.eye(32))))

3.722326266664133e-14


In [14]:
u = rom.solve([1.0, 0.1, 0.3, 0.1, 0.2, 1.0])
print(u)
U_red = reductor.reconstruct(u)
print(U_red.dim)

[[ 0.56005684  0.19436602 -0.19193242 -0.1265298  -0.1461029  -0.0533142
   0.10857938 -0.05102234  0.12284576  0.04276118  0.19080369  0.01323769
   0.02283033 -0.01018572  0.03660971 -0.01092604 -0.01132796 -0.01482546
   0.01353391 -0.01881186  0.00876463 -0.00155012  0.0105495   0.00950342
  -0.00552628  0.0054349  -0.012971   -0.00193113 -0.0027338  -0.00483226
   0.00109329  0.00181082]]
5101


In [15]:
ERR = U - U_red
print(ERR.norm(fom.h1_0_semi_product))
fom.visualize((U, U_red, ERR),
              legend=('Detailed', 'Reduced', 'Error'),
              separate_colorbars=True)

[0.00737445]


ThreeJSPlot(children=(HBox(children=(Renderer(children=(Renderer(camera=PerspectiveCamera(position=(0.0, 0.0, …